In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import joblib
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
# Load mood classification model
mood_model = load_model('../../models/mood_cnn_model.h5')

In [3]:
# Load music data
music_data = pd.read_csv('../../data/data_moods.csv')

In [4]:
# Feature selection
features = ['popularity', 'length', 'danceability', 'acousticness', 'energy', 
            'instrumentalness', 'liveness', 'valence', 'loudness', 
            'speechiness', 'tempo', 'key', 'time_signature']
X_music = music_data[features]
y_music = music_data['mood']

In [5]:
# Normalize features
scaler = StandardScaler()
X_music_scaled = scaler.fit_transform(X_music)

In [6]:
from sklearn.neighbors import NearestNeighbors
import joblib

In [7]:
# Train a NearestNeighbors model on music features
nn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
nn_model.fit(X_music_scaled)

NearestNeighbors(metric='cosine')

In [51]:
mood_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Function to recommend music based on mood
def recommend_music(mood_label):
    mood_map = {
        'happy': 'Happy',
        'sad': 'Sad',
        'fear': 'Sad',
        'surprise': 'Energetic',
        'angry': 'Energetic',
        'disgust': 'Energetic',
        'neutral': 'Calm'
    }
    
    mood_indices = [i for i, mood in enumerate(y_music) if mood == mood_map.get(mood_label)]
    if len(mood_indices) == 0:
        return pd.DataFrame()  # Return empty DataFrame if no music available for this mood
    recommended_indices = nn_model.kneighbors(X_music_scaled[mood_indices], return_distance=False)
    return music_data.iloc[recommended_indices[0]]

def load_and_preprocess_image(image_path, target_size=(48, 48)):
    img = image.load_img(image_path, target_size=target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

image_path = '../../data/test/angry/PrivateTest_88305.jpg'
processed_image = load_and_preprocess_image(image_path)
# Check prediction
prediction = mood_model.predict(processed_image)

# Convert prediction to mood label
predicted_mood = mood_labels[np.argmax(prediction)]
print("Predicted Mood:", predicted_mood)

# Get recommendations
recommendations = recommend_music(predicted_mood)
print("Recommendations:\n", recommendations[['name', 'artist', 'album']])


1/1 [==============================] - 0s 18ms/step
Predicted Mood: fear
Recommendations:
                                 name              artist  \
1                                 23      Blonde Redhead   
138                     Desire Lines                Lush   
461  Sahara Love - Seven Lions Remix      Above & Beyond   
626                     Welcome Home  Coheed and Cambria   
358                        Mudshovel              Staind   

                                                 album  
1                                                   23  
138                                              Split  
461                                        Sahara Love  
626  Good Apollo I'm Burning Star IV Volume One: Fr...  
358                                        Dysfunction  


In [ ]:
# Save the music recommendation model
joblib.dump(nn_model, '../../models/music_recommender.pkl')